In [11]:
//1. Модель классов (Entity Model)
using System;
using System.Collections.Generic;
using System.Linq;

// Enum для статусов заказа
public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}

// Класс "Заказчик"
public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string PhoneNumber { get; set; }

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }

    public override string ToString()
    {
        return $"{FirstName} {LastName}, Телефон: {PhoneNumber}";
    }
}

// Класс "Сотрудник"
public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
        OrdersProcessed = 0;
    }

    public override string ToString()
    {
        return $"{FirstName} {LastName} ({Position}), Заказы: {OrdersProcessed}";
    }
}


// Класс "Заказ"
public class Order
{
    public int OrderId { get; }
    public string Description { get; set; }
    public Employee AssignedEmployee { get; set; }
    public OrderStatus Status { get; set; }
    public DateTime CreatedAt { get; }
    public Customer Customer { get; set; } // Добавлено поле для заказчика

    // Конструктор для добавления заказчика
    public Order(int orderId, string description, Customer customer)
    {
        OrderId = orderId;
        Description = description;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
        Customer = customer;
    }

    // Конструктор для добавления заказа без заказчика
    public Order(int orderId, string description)
    {
        OrderId = orderId;
        Description = description;
        Status = OrderStatus.Created;
        CreatedAt = DateTime.Now;
    }

    public override string ToString()
    {
        string employeeInfo = AssignedEmployee != null ? AssignedEmployee.ToString() : "Не назначен";
        return $"ID: {OrderId}, Описание: {Description}, Статус: {Status}, Созданный: {CreatedAt}, Работник: {employeeInfo}, Покупатель: {Customer}";
    }
}


// Класс "Компания"
public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; } = new List<Employee>();
    public List<Order> Orders { get; } = new List<Order>();
    public List<Customer> Customers { get; } = new List<Customer>();


    // Делегат для уведомления об изменении статуса заказа
    public delegate void OrderStatusChangedHandler(Order order, string message);

    // Событие изменения статуса заказа
    public event OrderStatusChangedHandler OrderStatusChanged;

    private int _nextOrderId = 1; // Автоматическая нумерация заказов

    public Company(string name)
    {
        Name = name;
    }

    // Метод добавления сотрудника
    public void AddEmployee(Employee employee)
    {
        Employees.Add(employee);
    }

    // Метод добавления заказа
        public void AddOrder(Order order)
        {
            Orders.Add(order);
        }

    public void AddCustomer(Customer customer)
    {
        Customers.Add(customer);
    }

    public Order CreateOrder(string description, Customer customer)
    {
        var newOrder = new Order(_nextOrderId++, description, customer);
        Orders.Add(newOrder);
        return newOrder;
    }

    public Order CreateOrder(string description)
    {
        var newOrder = new Order(_nextOrderId++, description);
        Orders.Add(newOrder);
        return newOrder;
    }



    // Назначение сотрудника на заказ
    public void AssignEmployeeToOrder(Order order, Employee employee)
    {
        if (order.AssignedEmployee != null)
        {
            // Уменьшить количество обработанных заказов у предыдущего сотрудника (если есть)
            order.AssignedEmployee.OrdersProcessed--;
        }
        order.AssignedEmployee = employee;
        employee.OrdersProcessed++;
    }

    // Изменение статуса заказа
    public void ChangeOrderStatus(Order order, OrderStatus newStatus)
    {
        if (order.Status != newStatus)
        {
            order.Status = newStatus;
            OnOrderStatusChanged(order, $"Заказ {order.OrderId} статус изменен на {newStatus}");
        }
    }

    // Метод для вызова события
    protected virtual void OnOrderStatusChanged(Order order, string message)
    {
        OrderStatusChanged?.Invoke(order, message);
    }


    // Просмотр заказов по статусу
    public List<Order> GetOrdersByStatus(OrderStatus status)
    {
        return Orders.Where(o => o.Status == status).ToList();
    }

    // Поиск заказов по заказчику
    public List<Order> FindOrdersByCustomer(string customerFirstName, string customerLastName)
    {
        return Orders.Where(order =>
            order.Customer != null &&
            order.Customer.FirstName.Equals(customerFirstName, StringComparison.OrdinalIgnoreCase) &&
            order.Customer.LastName.Equals(customerLastName, StringComparison.OrdinalIgnoreCase)).ToList();
    }



    // Отчет по сотрудникам
    public void GenerateEmployeeReport()
    {
        Console.WriteLine("\n--- Отчет о сотруднике ---");
        foreach (var employee in Employees)
        {
            Console.WriteLine(employee);
        }
    }
}
//2. Логика приложения (Program.cs)

public class Program
{
    public static void Main(string[] args)
    {
        // Создаем компанию
        var company = new Company("Acme Corp");

        // Создаем сотрудников
        var employee1 = new Employee("Джон", "Доу", "Менеджер");
        var employee2 = new Employee("Джейн", "Смит", "Разработчик");
        company.AddEmployee(employee1);
        company.AddEmployee(employee2);


        // Создаем заказчиков
        var customer1 = new Customer("Иван", "Иванов", "7 (953) 164-15-61");
        var customer2 = new Customer("Боб", "Строитель", "7 (930) 063-18-75");

        company.AddCustomer(customer1);
        company.AddCustomer(customer2);



         // Создание заказов через методы компании, с заказчиками и без
        var order1 = company.CreateOrder("Создайте новый веб-сайт", customer1);
        var order2 = company.CreateOrder("Разработка мобильного приложения");
        var order3 = company.CreateOrder("Исправлены некоторые ошибки", customer2);

        // Назначение ответственных
        company.AssignEmployeeToOrder(order1, employee1);
        company.AssignEmployeeToOrder(order2, employee2);
        company.AssignEmployeeToOrder(order3, employee1);


        // Подписка на событие изменения статуса
        company.OrderStatusChanged += (order, message) =>
        {
            Console.WriteLine($"Уведомление: {message} (Идентификатор заказа: {order.OrderId})");
        };


         // Изменяем статусы заказов
        company.ChangeOrderStatus(order1, OrderStatus.InProgress);
        company.ChangeOrderStatus(order2, OrderStatus.Completed);
        company.ChangeOrderStatus(order3, OrderStatus.Completed);


         // Отображаем заказы по статусам
        Console.WriteLine("\n--- Выполняемые заказы ---");
        foreach (var order in company.GetOrdersByStatus(OrderStatus.InProgress))
        {
            Console.WriteLine(order);
        }

        Console.WriteLine("\n--- Выполненные заказы ---");
        foreach (var order in company.GetOrdersByStatus(OrderStatus.Completed))
        {
            Console.WriteLine(order);
        }


         // Поиск заказов по заказчику
        Console.WriteLine("\n--- Заказы на Иван Иванов ---");
        var foundOrders = company.FindOrdersByCustomer("Иван", "Иванов");
        foreach (var order in foundOrders)
        {
            Console.WriteLine(order);
        }

         // Отчет по сотрудникам
        company.GenerateEmployeeReport();


       // Вывод всех заказов
        Console.WriteLine("\n--- Все заказы ---");
        foreach (var order in company.Orders)
        {
             Console.WriteLine(order);
        }

         Console.WriteLine("\nНажмите любую клавишу для выхода.");
         Console.ReadKey();

    }
}
// Создаем компанию
        var company = new Company("Acme Corp");

        // Создаем сотрудников
        var employee1 = new Employee("Джон", "Доу", "Менеджер");
        var employee2 = new Employee("Джейн", "Смит", "Разработчик");
        company.AddEmployee(employee1);
        company.AddEmployee(employee2);


        // Создаем заказчиков
        var customer1 = new Customer("Иван", "Иванов", "7 (953) 164-15-61");
        var customer2 = new Customer("Боб", "Строитель", "7 (930) 063-18-75");

        company.AddCustomer(customer1);
        company.AddCustomer(customer2);



         // Создание заказов через методы компании, с заказчиками и без
        var order1 = company.CreateOrder("Создайте новый веб-сайт", customer1);
        var order2 = company.CreateOrder("Разработка мобильного приложения");
        var order3 = company.CreateOrder("Исправлены некоторые ошибки", customer2);

        // Назначение ответственных
        company.AssignEmployeeToOrder(order1, employee1);
        company.AssignEmployeeToOrder(order2, employee2);
        company.AssignEmployeeToOrder(order3, employee1);


        // Подписка на событие изменения статуса
        company.OrderStatusChanged += (order, message) =>
        {
            Console.WriteLine($"Уведомление: {message} (Идентификатор заказа: {order.OrderId})");
        };


         // Изменяем статусы заказов
        company.ChangeOrderStatus(order1, OrderStatus.InProgress);
        company.ChangeOrderStatus(order2, OrderStatus.Completed);
        company.ChangeOrderStatus(order3, OrderStatus.Completed);


         // Отображаем заказы по статусам
        Console.WriteLine("\n--- Выполняемые заказы ---");
        foreach (var order in company.GetOrdersByStatus(OrderStatus.InProgress))
        {
            Console.WriteLine(order);
        }

        Console.WriteLine("\n--- Выполненные заказы ---");
        foreach (var order in company.GetOrdersByStatus(OrderStatus.Completed))
        {
            Console.WriteLine(order);
        }


         // Поиск заказов по заказчику
        Console.WriteLine("\n--- Заказы на Иван Иванов ---");
        var foundOrders = company.FindOrdersByCustomer("Иван", "Иванов");
        foreach (var order in foundOrders)
        {
            Console.WriteLine(order);
        }

         // Отчет по сотрудникам
        company.GenerateEmployeeReport();


       // Вывод всех заказов
        Console.WriteLine("\n--- Все заказы ---");
        foreach (var order in company.Orders)
        {
             Console.WriteLine(order);
        }

         Console.WriteLine("\nНажмите любую клавишу для выхода.");

Уведомление: Заказ 1 статус изменен на InProgress (Идентификатор заказа: 1)
Уведомление: Заказ 2 статус изменен на Completed (Идентификатор заказа: 2)
Уведомление: Заказ 3 статус изменен на Completed (Идентификатор заказа: 3)

--- Выполняемые заказы ---
ID: 1, Описание: Создайте новый веб-сайт, Статус: InProgress, Созданный: 12/21/2024 12:56:20 PM, Работник: Джон Доу (Менеджер), Заказы: 2, Покупатель: Иван Иванов, Телефон: 7 (953) 164-15-61

--- Выполненные заказы ---
ID: 2, Описание: Разработка мобильного приложения, Статус: Completed, Созданный: 12/21/2024 12:56:20 PM, Работник: Джейн Смит (Разработчик), Заказы: 1, Покупатель: 
ID: 3, Описание: Исправлены некоторые ошибки, Статус: Completed, Созданный: 12/21/2024 12:56:20 PM, Работник: Джон Доу (Менеджер), Заказы: 2, Покупатель: Боб Строитель, Телефон: 7 (930) 063-18-75

--- Заказы на Иван Иванов ---
ID: 1, Описание: Создайте новый веб-сайт, Статус: InProgress, Созданный: 12/21/2024 12:56:20 PM, Работник: Джон Доу (Менеджер), Заказы: